In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

train_path = "archive/My_data/train"
img_size = (150, 150)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

train_data = datagen.flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False 

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(
    train_data,
    validation_data=val_data,
    epochs=25,
    callbacks=[early_stop]
)

model.save("fruit_model_mobilenet.h5")


Found 1872 images belonging to 12 classes.
Found 467 images belonging to 12 classes.
Epoch 1/25
59/59 [==============================] - 24s 372ms/step - loss: 1.3918 - accuracy: 0.5668 - val_loss: 0.6843 - val_accuracy: 0.7901
Epoch 2/25
59/59 [==============================] - 23s 393ms/step - loss: 0.7097 - accuracy: 0.7826 - val_loss: 0.5749 - val_accuracy: 0.8373
Epoch 3/25
59/59 [==============================] - 24s 405ms/step - loss: 0.5834 - accuracy: 0.8109 - val_loss: 0.6274 - val_accuracy: 0.8501
Epoch 4/25
59/59 [==============================] - 23s 395ms/step - loss: 0.5280 - accuracy: 0.8397 - val_loss: 0.5713 - val_accuracy: 0.8480
Epoch 5/25
59/59 [==============================] - 24s 401ms/step - loss: 0.4571 - accuracy: 0.8643 - val_loss: 0.5632 - val_accuracy: 0.8587
Epoch 6/25
59/59 [==============================] - 23s 392ms/step - loss: 0.4445 - accuracy: 0.8590 - val_loss: 0.5220 - val_accuracy: 0.8715
Epoch 7/25
59/59 [==============================] - 24s 4

In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2

model = tf.keras.models.load_model("fruit_model_mobilenet.h5")

predict_dir = r"C:\Users\pruth\Downloads\VIT Downloads\Self_Checkout\archive\MY_data\predict"
train_dir = r"C:\Users\pruth\Downloads\VIT Downloads\Self_Checkout\archive\MY_data\train"
image_size = (150, 150)

class_labels = sorted(os.listdir(train_dir))

for item in os.listdir(predict_dir):
    item_path = os.path.join(predict_dir, item)

    if os.path.isdir(item_path):
        for img_file in os.listdir(item_path):
            img_path = os.path.join(item_path, img_file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, image_size)
            img = img.astype('float32') / 255.0
            img = np.expand_dims(img, axis=0)

            prediction = model.predict(img)
            predicted_index = np.argmax(prediction)
            predicted_label = class_labels[predicted_index]
            confidence = np.max(prediction)

            print(f"[{img_file}] → Predicted: {predicted_label}, Confidence: {confidence:.2f}")

    elif os.path.isfile(item_path):
        img = cv2.imread(item_path)
        if img is None:
            continue
        img = cv2.resize(img, image_size)
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=0)

        prediction = model.predict(img)
        predicted_index = np.argmax(prediction)
        predicted_label = class_labels[predicted_index]
        confidence = np.max(prediction)

        print(f"[{item}] → Predicted: {predicted_label}, Confidence: {confidence:.2f}")


1/1 [==============================] - 1s 637ms/step
[00.jpeg] → Predicted: cherry, Confidence: 0.27
1/1 [==============================] - 0s 33ms/step
[0122.jpeg] → Predicted: watermelon, Confidence: 0.80
1/1 [==============================] - 0s 29ms/step
[22.jpeg] → Predicted: pinenapple, Confidence: 0.94
1/1 [==============================] - 0s 30ms/step
[3.jpeg] → Predicted: avocado, Confidence: 0.50
1/1 [==============================] - 0s 35ms/step
[321.jpeg] → Predicted: kiwi, Confidence: 0.98
1/1 [==============================] - 0s 33ms/step
[33.jpeg] → Predicted: orange, Confidence: 0.40
1/1 [==============================] - 0s 29ms/step
[365.jpeg] → Predicted: predict, Confidence: 0.47
1/1 [==============================] - 0s 30ms/step
[4.jpeg] → Predicted: Apple, Confidence: 0.86
1/1 [==============================] - 0s 31ms/step
[41.jpeg] → Predicted: kiwi, Confidence: 0.66
1/1 [==============================] - 0s 29ms/step
[65.jpeg] → Predicted: cherry, Confidenc